<a href="https://colab.research.google.com/github/cxbxmxcx/CalgaryDeepLearning/blob/master/DRL_TD_Taxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[box2d] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (45.1.0)


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from tqdm import tqdm
from IPython.display import HTML

from IPython import display as ipythondisplay

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1009'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [0]:
from gym import envs
print(envs.registry.all())

dict_values([EnvSpec(Copy-v0), EnvSpec(RepeatCopy-v0), EnvSpec(ReversedAddition-v0), EnvSpec(ReversedAddition3-v0), EnvSpec(DuplicatedInput-v0), EnvSpec(Reverse-v0), EnvSpec(CartPole-v0), EnvSpec(CartPole-v1), EnvSpec(MountainCar-v0), EnvSpec(MountainCarContinuous-v0), EnvSpec(Pendulum-v0), EnvSpec(Acrobot-v1), EnvSpec(LunarLander-v2), EnvSpec(LunarLanderContinuous-v2), EnvSpec(BipedalWalker-v2), EnvSpec(BipedalWalkerHardcore-v2), EnvSpec(CarRacing-v0), EnvSpec(Blackjack-v0), EnvSpec(KellyCoinflip-v0), EnvSpec(KellyCoinflipGeneralized-v0), EnvSpec(FrozenLake-v0), EnvSpec(FrozenLake8x8-v0), EnvSpec(CliffWalking-v0), EnvSpec(NChain-v0), EnvSpec(Roulette-v0), EnvSpec(Taxi-v3), EnvSpec(GuessingGame-v0), EnvSpec(HotterColder-v0), EnvSpec(Reacher-v2), EnvSpec(Pusher-v2), EnvSpec(Thrower-v2), EnvSpec(Striker-v2), EnvSpec(InvertedPendulum-v2), EnvSpec(InvertedDoublePendulum-v2), EnvSpec(HalfCheetah-v2), EnvSpec(HalfCheetah-v3), EnvSpec(Hopper-v2), EnvSpec(Hopper-v3), EnvSpec(Swimmer-v2), EnvSp

In [0]:
env = gym.make("Taxi-v3")

In [0]:
#check out the pacman action space!
print(env.action_space)

Discrete(6)


Setup

In [0]:
action_size = env.action_space.n
print("Action size ", action_size)

state_size = env.observation_space.n
print("State size ", state_size)

qtable = np.zeros((state_size, action_size))
print(qtable)

total_episodes = 50000 # Total episodes
total_test_episodes = 100 # Total test episodes
max_steps = 99 # Max steps per episode

learning_rate = 0.7 # Learning rate
gamma = 0.618 # Discounting rate

# Exploration parameters
epsilon = 1.0 # Exploration rate
max_epsilon = 1.0 # Exploration probability at start
min_epsilon = 0.01 # Minimum exploration probability 
decay_rate = 0.01 # Exponential decay rate for exploration prob

Action size  6
State size  500
[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


Learn and update Q table

In [0]:
# 2 For life or until learning is stopped
for episode in tqdm(range(total_episodes)):
# Reset the environment  
    try:  
      state = env.reset() 
    except:
      continue     
    step = 0
    done = False

    for step in range(max_steps):
# 3. Choose an action a in the current world state (s)
## First we randomize a number        
        exp_exp_tradeoff = random.uniform(0,1)

## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])

# Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

# Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)

# Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * 
        np.max(qtable[new_state, :]) - qtable[state, action])

# Our new state is state
        state = new_state

# If done : finish episode
        if done == True:
          state = env.reset()
          break
# Reduce epsilon (because we need less and less exploration)
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)

#env.reset()
rewards = []
print(qtable)



  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 28/50000 [00:00<02:58, 279.60it/s]

  0%|          | 59/50000 [00:00<02:54, 286.51it/s]

  0%|          | 93/50000 [00:00<02:46, 300.39it/s]

  0%|          | 132/50000 [00:00<02:35, 321.28it/s]

  0%|          | 178/50000 [00:00<02:21, 351.87it/s]

  0%|          | 238/50000 [00:00<02:03, 401.66it/s]

  1%|          | 340/50000 [00:00<01:41, 490.80it/s]

  1%|          | 550/50000 [00:00<01:17, 637.21it/s]

  2%|▏         | 787/50000 [00:00<01:00, 816.02it/s]

  2%|▏         | 1050/50000 [00:01<00:47, 1028.55it/s]

  3%|▎         | 1335/50000 [00:01<00:38, 1271.98it/s]

  3%|▎         | 1619/50000 [00:01<00:31, 1523.56it/s]

  4%|▎         | 1868/50000 [00:01<00:27, 1723.81it/s]

  4%|▍         | 2155/50000 [00:01<00:24, 1958.39it/s]

  5%|▍         | 2444/50000 [00:01<00:21, 2166.59it/s]

  5%|▌         | 2710/50000 [00:01<00:20, 2292.31it/s]

  6%|▌         | 2974/50000 [00:01<00:20, 2350.17it/s]

  6%|▋         | 3243/50

[[  0.           0.           0.           0.           0.
    0.        ]
 [ -2.50421533  -2.43428638  -2.50422073  -2.43402249  -2.32039715
  -11.43400513]
 [ -1.8391098   -1.35778803  -2.30820776  -1.36455096  -0.57891593
  -10.35771589]
 ...
 [ -1.97108429   0.67710243  -2.06537874  -2.09991323  -7.
   -9.1       ]
 [ -2.37869947  -2.13656935  -2.34693787  -2.40854779 -10.89526101
  -11.08068059]
 [ -1.21282     -1.21282     -0.91        11.36        -9.1
   -7.        ]]


Evaluate policy

In [0]:
env = wrap_env(gym.make("Taxi-v3"))

for episode in range(total_test_episodes):
    try:
      state = env.reset()
    except:
      continue
    step = 0
    done = False
    total_rewards = 0
    print("EPISODE ", episode)

    for step in range(max_steps):      
      # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])

        new_state, reward, done, info = env.step(action)
        env.render()
        total_rewards += reward

        if done:
            rewards.append(total_rewards)
            print ("Score", total_rewards)
            break
        state = new_state

env.close()
print ("Score over time: " + str(sum(rewards)/total_test_episodes))
show_video()

EPISODE  0
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
|_| : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
|_: : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : :_: : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : |_: : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : :_|
| : : : : |
| | 

In [0]:
!ls

openaigym.episode_batch.6.124.stats.json
openaigym.manifest.6.124.manifest.json
openaigym.video.6.124.video000000.json
openaigym.video.6.124.video000000.meta.json
openaigym.video.6.124.video000001.json
openaigym.video.6.124.video000001.meta.json
openaigym.video.6.124.video000008.json
openaigym.video.6.124.video000008.meta.json
openaigym.video.6.124.video000027.json
openaigym.video.6.124.video000027.meta.json
openaigym.video.6.124.video000064.json
openaigym.video.6.124.video000064.meta.json
video
